In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

In [ ]:
#| export
import sys

import typer
from rich.console import Console

from tui_writer.audio import AudioRecorder
from tui_writer.transcription import WhisperTranscriber
from tui_writer.tui import TranscriptionTUI
from tui_writer.utils import copy_to_clipboard

# Console for rich formatting
console = Console()



/home/mario/miniconda3/envs/tui_writer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CLI - Command Line Interface

This module provides both a TUI (Terminal User Interface) and CLI commands for audio transcription using Whisper models.

**Features:**
- Interactive TUI with real-time recording
- Model selection dialog
- Keyboard shortcuts for efficient workflow
- Auto-start recording on launch
- Clipboard integration



## Usage

### Launch the TUI (Default)

```bash
tui_writer
```

Recording starts automatically. Press Enter to stop and transcribe, or use keyboard shortcuts:
- `R` - Start recording
- `S` - Stop recording
- `T` - Transcribe last recording
- `L` - List/select models
- `C` - Copy to clipboard
- `Q` - Quit

### Explicit TUI Launch

```bash
python -m tui_writer.cli tui
```

### Transcribe Last Recording

```bash
python -m tui_writer.cli transcribe_last
```

## Commands


In [ ]:

#| export
app = typer.Typer()

@app.command()
def tui():
    """Launch the interactive TUI for audio transcription."""
    tui_app = TranscriptionTUI()
    tui_app.run()

@app.command()
def transcribe_last():
    """Transcribe the most recently recorded audio file."""
    try:
        recorder = AudioRecorder()
        audio_file_path = recorder._get_audio_file_path()
        if not audio_file_path.exists():
            console.print(
                "❌ [bold red]No previous recording found. Record audio first by running 'hns' without --last flag.[/bold red]"
            )
            sys.exit(1)

        transcriber = WhisperTranscriber()
        transcription, elapsed_time = transcriber.transcribe(audio_file_path)

        copy_to_clipboard(transcription, elapsed_time)

    except (RuntimeError, ValueError) as e:
        console.print(f"❌ [bold red]{e}[/bold red]")
        sys.exit(1)
    except Exception as e:
        console.print(f"❌ [bold red]Unexpected error: {e}[/bold red]")
        sys.exit(1)

